# Data Scraping

In this notebook, we will fetch and download all the necessary datasets and store them in their associated files within the "data" file

### Importing Libraries and Functions

In [1]:
import sys
sys.path.append('../')
import os
from json import dump
from scripts.datascrape_functions import generate_url_list, fetch_rental_data
from scripts.scrape_sa2 import get_shapefile

### Defining Global Variables - changes pending

In [2]:
# CONSTANTS
BASE_URL = "https://www.domain.com.au"
LANDING_PATH = "data/landing"
RAW_PATH = "data/raw"

### Fetching the '*domain.com*' Rental Property Data

In [3]:
# Generating the list of urls to fetch data from
url_links = generate_url_list(BASE_URL)



Generating the list of links...


Fetching listings for price range: 150-200

Found 57 listings for price range 150-200

Fetching listings for sub-range: 150-200

Visiting https://www.domain.com.au/rent/?price=150-200&excludedeposittaken=1&sort=price-asc&state=vic&page=1
Visiting https://www.domain.com.au/rent/?price=150-200&excludedeposittaken=1&sort=price-asc&state=vic&page=2
Visiting https://www.domain.com.au/rent/?price=150-200&excludedeposittaken=1&sort=price-asc&state=vic&page=3
Visiting https://www.domain.com.au/rent/?price=150-200&excludedeposittaken=1&sort=price-asc&state=vic&page=4
No listings found on page 4 for price range 150-200.

Fetching listings for price range: 200-250

Found 72 listings for price range 200-250

Fetching listings for sub-range: 200-250

Visiting https://www.domain.com.au/rent/?price=200-250&excludedeposittaken=1&sort=price-asc&state=vic&page=1
Visiting https://www.domain.com.au/rent/?price=200-250&excludedeposittaken=1&sort=price-asc&state=vic&page=2

In [4]:
print(len(url_links))

13838


In [ ]:
# Fetching the rental data from the domain.comm urls
property_metadata = fetch_rental_data(url_links)

In [4]:
# Creates the directory for the rental data if it doesn't exist
if not os.path.exists(f"../{LANDING_PATH}"):
    os.makedirs(f"../{LANDING_PATH}")

# Save the data in the data/landing directory
with open(f'../{LANDING_PATH}/testing_scraping.json', 'w') as f:
    dump(property_metadata, f)
    print(f"File saved in the {LANDING_PATH} directory")


File saved in the data/landing directory


In [3]:
# scrapes SA2 shapfile and downloads to data folder

get_shapefile(url="https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip", output_dir = '../data/SA2/')